# Research In-Lab Three

### Name:  Chethana Chennuri   


### Date: 10/17/2022


### PID: 730 398 710


### Jupyter Notebook Number: 10

## Purpose/Objective:
Group Objective: Perform calculations through the unpaired t-test for averages of firing rates.

Indiviudal Objective: Create a spike histogram

## Protocol:

import Allen Institute data set

define units of interest for ids

create a waveforms variable to map out membrane potential over time of each unit



In [1]:
pip install allensdk

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Import Packages:

In [2]:
import os
import shutil
import allensdk
import pprint
from pathlib import Path

import numpy as np
import pandas as pd
import scipy.stats as st

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('notebook', font_scale=1.5, rc={'lines.markeredgewidth': 2})

### Import Visual Coding Data

In [7]:
#this code block should only be run if you are working with the neuropixels data
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

data_directory = '/overflow/NSCI274/projects/ecephysdata/' 

manifest_path = os.path.join(data_directory, "manifest.json")

from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot

from allensdk.brain_observatory.visualization import plot_running_speed

cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

### Print Table of All Data

In [10]:
sessions = cache.get_session_table()

print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


### Waveforms

In [26]:
session.stimulus_presentations.head()

,color,contrast,frame,orientation,phase,size,spatial_frequency,start_time,stimulus_block,stimulus_name,stop_time,temporal_frequency,x_position,y_position,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,,,,,
0,null,null,null,null,null,null,null,24.429348,null,spontaneous,84.496188,null,null,null,60.066840,0
1,null,0.8,null,45.0,"[3644.93333333, 3644.93333333]","[20.0, 20.0]",0.08,84.496188,0.0,gabors,84.729704,4.0,40.0,30.0,0.233516,1
2,null,0.8,null,45.0,"[3644.93333333, 3644.93333333]","[20.0, 20.0]",0.08,84.729704,0.0,gabors,84.979900,4.0,-30.0,10.0,0.250196,2
3,null,0.8,null,90.0,"[3644.93333333, 3644.93333333]","[20.0, 20.0]",0.08,84.979900,0.0,gabors,85.230095,4.0,10.0,-10.0,0.250196,3
4,null,0.8,null,90.0,"[3644.93333333, 3644.93333333]","[20.0, 20.0]",0.08,85.230095,0.0,gabors,85.480291,4.0,30.0,40.0,0.250196,4


In [28]:
session.stimulus_names

['spontaneous',
 'gabors',
 'flashes',
 'drifting_gratings',
 'natural_movie_three',
 'natural_movie_one',
 'static_gratings',
 'natural_scenes']

In [41]:
natural_scenes_sessions = session.get_stimulus_table(['natural_scenes']).head()

In [32]:
session.get_stimulus_table(['natural_movie_one']).head()

,contrast,frame,orientation,size,start_time,stimulus_block,stimulus_name,stop_time,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,
22000,1.0,0.0,0.0,"[1920.0, 1080.0]",2846.788598,4.0,natural_movie_one,2846.821959,0.033361,3887
22001,1.0,1.0,0.0,"[1920.0, 1080.0]",2846.821959,4.0,natural_movie_one,2846.855320,0.033361,3888
22002,1.0,2.0,0.0,"[1920.0, 1080.0]",2846.855320,4.0,natural_movie_one,2846.888681,0.033361,3889
22003,1.0,3.0,0.0,"[1920.0, 1080.0]",2846.888681,4.0,natural_movie_one,2846.922042,0.033361,3890
22004,1.0,4.0,0.0,"[1920.0, 1080.0]",2846.922042,4.0,natural_movie_one,2846.955403,0.033361,3891


In [36]:
session.get_stimulus_table(['natural_movie_three']).head()

,contrast,frame,orientation,size,start_time,stimulus_block,stimulus_name,stop_time,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,
3999,1.0,0.0,0.0,"[1920.0, 1080.0]",2216.261498,3.0,natural_movie_three,2216.294859,0.033361,287
4000,1.0,1.0,0.0,"[1920.0, 1080.0]",2216.294859,3.0,natural_movie_three,2216.328220,0.033361,288
4001,1.0,2.0,0.0,"[1920.0, 1080.0]",2216.328220,3.0,natural_movie_three,2216.361581,0.033361,289
4002,1.0,3.0,0.0,"[1920.0, 1080.0]",2216.361581,3.0,natural_movie_three,2216.394942,0.033361,290
4003,1.0,4.0,0.0,"[1920.0, 1080.0]",2216.394942,3.0,natural_movie_three,2216.428303,0.033361,291


In [42]:
#session_id = 756029989
session = cache.get_session_data(natural_scenes_sessions)

TypeError: %d format: a number is required, not DataFrame

In [38]:
print(f"session {session.ecephys_session_id} was acquired in {session.session_start_time}")

session 756029989 was acquired in 2018-10-26 12:59:18-07:00


In [39]:
session.units.head()

,waveform_PT_ratio,waveform_amplitude,amplitude_cutoff,cluster_id,cumulative_drift,d_prime,firing_rate,isi_violations,isolation_distance,L_ratio,...,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,
951814884,0.522713,187.434780,0.042404,6,463.86,3.555518,9.492176,0.181638,46.750473,0.024771,...,215.0,APN,8162.0,3487.0,6737.0,probeA,See electrode locations,29999.949611,1249.9979,True
951814876,0.652514,129.686505,0.097286,5,325.21,4.445414,39.100557,0.004799,85.178750,0.001785,...,215.0,APN,8162.0,3487.0,6737.0,probeA,See electrode locations,29999.949611,1249.9979,True
951815032,0.484297,207.380940,0.015482,17,396.28,3.848256,28.383277,0.007099,89.608836,0.035654,...,215.0,APN,8129.0,3419.0,6762.0,probeA,See electrode locations,29999.949611,1249.9979,True
951815275,0.600600,158.158650,0.063807,30,374.82,3.065938,5.709358,0.032317,48.114336,0.016783,...,215.0,APN,8095.0,3349.0,6787.0,probeA,See electrode locations,29999.949611,1249.9979,True
951815314,0.459025,173.475705,0.072129,34,420.05,4.198612,23.902235,0.048075,76.916334,0.009666,...,215.0,APN,8088.0,3333.0,6792.0,probeA,See electrode locations,29999.949611,1249.9979,True


In [35]:
print(f'{sessions.units.shape[0]} units total')
units_with_very_high_snr = sessions.units[session.units['snr'] > 4]
print(f'{units_with_very_high_snr.shape[0]} units have snr > 4')

AttributeError: 'DataFrame' object has no attribute 'units'

In [8]:
units_of_interest = high_snr_unit_ids[:35]

waveforms = {uid: session.mean_waveforms[uid] for uid in units_of_interest}
peak_channels = {uid: session.units.loc[uid, 'peak_channel_id'] for uid in units_of_interest}

# plot the mean waveform on each unit's peak channel/
plot_mean_waveforms(waveforms, units_of_interest, peak_channels)
plt.show()

NameError: name 'high_snr_unit_ids' is not defined

### Website for Reference

https://www.pythonfordatascience.org/independent-samples-t-test-python/